In [1]:
import pandas as pd

In [2]:
def compute_lf(v_actual, v_max, engine='main', type='container', status='trip'):
    """
    Compute the load factor (LF) of an aircraft.

    Parameters:
    v_actual (float): Actual airspeed of the aircraft.
    v_max (float): Maximum airspeed of the aircraft.
    engine (str): Type of engine ('main' or 'auxiliary').
    type (str): Type of aircraft (e.g., 'container', 'bulk_carrier').
    status (str): Operational status ('trip' or 'maneuver').

    Returns:
    float: Load factor (LF).
    """

    if engine == 'main':
        lf = (v_actual / v_max) ** 3
        return lf
    elif engine == 'auxiliary':
        try:
            lf_table = pd.read_csv('data/LF_auxiliary.csv')
            
            row = lf_table[lf_table['ship'] == type]
            
            if row.empty:
                print(f"Warning: Can not find ship type '{type}'. Default LF=0.5 will be used.")
                return 0.5
            
            if status in row.columns:
                lf = row[status].values[0]
                return float(lf)
            else:
                raise ValueError(f"Status '{status}' is invalid (only accept: trip, maneuver, mooring)")
                
        except FileNotFoundError:
            raise FileNotFoundError("File 'LF_auxiliary.csv' not found")
            
    else:
        raise ValueError("Engine must be 'main' or 'auxiliary'")


print("Test 1 - Máy chính (Container, chạy 20/25 knot):")
print(compute_lf(20, 25, engine='main'))

print("\nTest 2 - Máy phụ (Tàu du lịch, chế độ mooring):")
print(compute_lf(0, 0, engine='auxiliary', type='cruise_ship', status='mooring'))



Test 1 - Máy chính (Container, chạy 20/25 knot):
0.5120000000000001

Test 2 - Máy phụ (Tàu du lịch, chế độ mooring):
0.64


In [3]:
def compute_ef_base(pollutants, engine='main', engine_speed='SSD', year=2020, tier=0, rpm=150):
    """
    Compute the Base Emission Factor (EF_base) for a list of pollutants.

    Parameters:
    pollutants (list): List of pollutant types (e.g., ['NOx', 'CO2']).
    engine (str): Type of engine ('main' or 'auxiliary').
    engine_speed (str): Speed category for main engines ('SSD', 'MSD', 'HSD').
    year (int): Year of the ship's construction.
    tier (int): Tier standard of the engine (0, 1, 2, 3).
    rpm (int): Revolutions per minute of the engine.

    Returns:
    dict: A dictionary mapping each pollutant to its Base Emission Factor (g/kWh).
    """


    match year:
        case year if year < 2000:
            tier = 0    
        case year if 2000 <= year < 2009:
            tier = 1
        case year if 2009 <= year < 2016:
            tier = 2
        case year if year >= 2016:
            tier = 3
        case _:
            raise ValueError("Year is invalid")

    match rpm:
        case rpm if rpm < 130:
            engine_speed = 'SSD'
        case rpm if 130 <= rpm < 2000:
            engine_speed = 'MSD'
        case rpm if rpm >= 2000:    
            engine_speed = 'HSD'
        case _:
            raise ValueError("RPM is invalid")


    try:
        ef_table = pd.read_csv('data/EF_base.csv')
    except FileNotFoundError:
        raise FileNotFoundError("File 'EF_base.csv' not found")

    results = {}


    for pollutant in pollutants:

        row = ef_table[(ef_table['exhaust'] == pollutant) & (ef_table['Tier'] == tier)]

        if row.empty:
            print(f"Warning: Configuration for pollutant '{pollutant}' at Tier {tier} not found. Returning 0.0.")
            results[pollutant] = 0.0
            continue

        ef_value = 0.0

        if engine == 'main':
            if engine_speed == 'SSD':

                ef_value = row['main (SSD)'].values[0]
            elif engine_speed == 'MSD':

                ef_value = row['main(MSD)'].values[0]
            elif engine_speed == 'HSD':
               
                ef_value = row['main(HSD)'].values[0]
            else:
                raise ValueError(f"Engine speed '{engine_speed}' is invalid (only accept: SSD, MSD)")
        
        elif engine == 'auxiliary':
            
            ef_value = row['auxiliary'].values[0]
        
        else:
            raise ValueError("Engine must be 'main' or 'auxiliary'")
        
        results[pollutant] = float(ef_value)

    return results


print("Test 1 - Main Engine (Year < 2000 -> Tier 0, RPM < 130 -> SSD):")
# Input: List of pollutants
# Expected: {'NOx': 17.0, 'CO2': 588.0}
print(compute_ef_base(['NOx', 'CO2'], engine='main', year=1995, rpm=100))

print("\nTest 2 - Auxiliary Engine (Year 2010 -> Tier 2):")
# Expected: {'NOx': 11.0} (value from 'sub' column for Tier 2 NOx)
print(compute_ef_base(['NOx'], engine='auxiliary', year=2010))



Test 1 - Main Engine (Year < 2000 -> Tier 0, RPM < 130 -> SSD):
{'NOx': 17.0, 'CO2': 593.0}

Test 2 - Auxiliary Engine (Year 2010 -> Tier 2):
{'NOx': 10.5}


In [4]:
import pandas as pd
import numpy as np

def compute_lla(pollutants, lf):
    """
    Tính hệ số điều chỉnh tải thấp (Low Load Adjustment - LLA) cho động cơ Non-MAN.
    
    Parameters:
    pollutants (list): Danh sách các chất ô nhiễm (ví dụ: ['NOx', 'HC']).
    lf (float): Hệ số tải thực tế (0.0 - 1.0).
    
    Returns:
    dict: Dictionary chứa hệ số LLA cho từng chất ô nhiễm.
    """
    try:
        lla_table = pd.read_csv('data/LLA_non_man.csv')
    except FileNotFoundError:
        print("Error: File 'LLA_non_man.csv' not found.")
        return {p: 1.0 for p in pollutants}

    if lla_table['Load'].dtype == object:
        lla_table['Load'] = lla_table['Load'].str.replace('%', '').astype(float)
        
    results = {}
    


    current_load_percent = lf * 100
    max_table_load = lla_table['Load'].max() 

    for pollutant in pollutants:
    
        col_name = pollutant

        if col_name not in lla_table.columns:
            print(f"Warning: Pollutant '{pollutant}' (mapped to '{col_name}') not found in LLA columns. Returning 1.0.")
            results[pollutant] = 1.0
            continue

        if current_load_percent > max_table_load:
            results[pollutant] = 1.0
        else:
            nearest_idx = (lla_table['Load'] - current_load_percent).abs().idxmin()
            
            val = lla_table.loc[nearest_idx, col_name]
            results[pollutant] = float(val)

    return results


print("Test 1: Tải rất thấp (2%) cho NOx và HC:")
print(compute_lla(['NOx', 'HC'], lf=0.025))

print("\nTest 2: Tải cao (50%) - vượt bảng:")
print(compute_lla(['CO2'], lf=0.5))

Test 1: Tải rất thấp (2%) cho NOx và HC:
{'NOx': 4.63, 'HC': 21.18}

Test 2: Tải cao (50%) - vượt bảng:
{'CO2': 1.0}


In [5]:
import pandas as pd

def compute_efa_non_man(pollutants, valve_type='C3'):
    """
    Compute Emission Factor Adjustment (EFA) for Non-MAN engines.
    
    Based on Formula: Adjusted EF = EF_base * EFA
    This function returns the EFA component from Table 4.

    Parameters:
    pollutants (list): List of pollutant types (e.g., ['NOx', 'CO', 'HC']).
    valve_type (str): Type of valve configuration. 
                      'SV' (Slide Valve) or 'C3' (No Slide Valve / Conventional).
                      Default is 'C3'.

    Returns:
    dict: Dictionary mapping pollutants to their EFA values.
    """
    
    # 1. Load the lookup table
    try:
        df = pd.read_csv('data/EFA_non_man.csv')
    except FileNotFoundError:
        raise FileNotFoundError("File 'EFA_non_man.csv' not found")

    # 2. Validate valve_type (Column selection)
    if valve_type not in ['SV', 'C3']:
        raise ValueError(f"Valve type '{valve_type}' is invalid. Use 'SV' (Slide Valve) or 'C3' (No Slide Valve).")

    # 3. Column mapping (User input -> CSV Pollutant name)
    # Mapping HC to VOC, and PM10/PM2.5 to generic PM as per Table 4

    results = {}

    for p in pollutants:
        # Map input name to table name
        table_name = p
        
        # Filter row by Pollutant name
        row = df[df['Pollutant'] == table_name]
        
        if row.empty:
            print(f"Warning: Pollutant '{p}' (mapped to '{table_name}') not found in EFA table. Returning 1.0.")
            results[p] = 1.0
        else:
            # Get value from the specific column (SV or C3)
            val = row[valve_type].values[0]
            results[p] = float(val)

    return results


# --- TEST CASES ---

print("Test 1 - CO for Engine with Slide Valve (SV):")
# Expected: 0.59
print(compute_efa_non_man(['CO'], valve_type='SV'))

print("\nTest 2 - CO for Engine without Slide Valve (C3):")
# Expected: 0.44
print(compute_efa_non_man(['CO'], valve_type='C3'))

print("\nTest 3 - NOx and HC (mapped to VOC) for C3:")
# Expected: NOx: 1.0, HC: 1.0
print(compute_efa_non_man(['NOx', 'HC'], valve_type='C3'))

Test 1 - CO for Engine with Slide Valve (SV):
{'CO': 0.59}

Test 2 - CO for Engine without Slide Valve (C3):
{'CO': 0.44}

Test 3 - NOx and HC (mapped to VOC) for C3:
{'NOx': 1.0, 'HC': 1.0}


In [6]:
def compute_real_ef_non_man(pollutants, lf, engine='main', year=2010, rpm=100, valve_type='C3'):
    """
    Compute the Real Emission Factor (EF) for Non-MAN engines.
    Formula: EF = EF_base * EFA * LLA (Low Load Adjustment)
    """
    
    base_efs = compute_ef_base(pollutants, engine='main', year=year, rpm=rpm)
    
    if lf > 0.2:
        real_ef = compute_ef_base(pollutants, engine='main', year=year, rpm=rpm)
        print( real_ef)
        return real_ef

    efas = compute_efa_non_man(pollutants, valve_type=valve_type)
    

    llas = compute_lla(pollutants, lf)
    
    final_ef = {}

    for p in pollutants:
        b = base_efs.get(p, 0.0)
        e = efas.get(p, 1.0)
        l = llas.get(p, 1.0)

        real = b * e * l
        final_ef[p] = real
        
    print( final_ef)
        
    return final_ef


# --- TEST CASES ---

# Trường hợp 1: Tàu chạy tải thấp (2%), Máy Non-MAN C3, Năm 2010 (Tier 2)
print("TEST 1: Low Load (2%), Tier 2, C3 Valve")
ef_result = compute_real_ef_non_man(['NOx', 'CO2'], lf=0.02, year=2010, rpm=100, valve_type='C3')

# Trường hợp 2: Tàu chạy tải bình thường (50%), Máy Non-MAN SV, Năm 2010
print("\nTEST 2: Normal Load (50%), Tier 2, Slide Valve (SV)")
ef_result_2 = compute_real_ef_non_man(['NOx', 'CO'], lf=0.50, year=2010, rpm=100, valve_type='SV')

TEST 1: Low Load (2%), Tier 2, C3 Valve
{'NOx': 66.672, 'CO2': 1945.04}

TEST 2: Normal Load (50%), Tier 2, Slide Valve (SV)
{'NOx': 14.4, 'CO': 1.4}


In [7]:
def compute_laf_man(pollutants, lf, man_type='C3'):
    """
    Tính hệ số điều chỉnh tải (LAF) cho động cơ MAN.
    Động cơ MAN thường có bảng tra chi tiết từ 1% - 100% tải.

    Parameters:
    pollutants (list): Danh sách chất ô nhiễm.
    lf (float): Hệ số tải (0.0 - 1.0).
    man_type (str): Loại cấu hình MAN ('C3' hoặc 'SV').

    Returns:
    dict: Dictionary chứa LAF cho từng chất.
    """
    
    # 1. Xác định file dữ liệu dựa trên loại MAN
    if man_type == 'C3':
        filename = 'data/LAF_MAN_C3.csv'
    elif man_type == 'SV':
        filename = 'data/LAF_MAN_SV.csv'
    else:
        raise ValueError("Loại MAN không hợp lệ. Chỉ chấp nhận 'C3' hoặc 'SV'.")

    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        raise FileNotFoundError(f"Không tìm thấy file '{filename}'")

    # 2. Xử lý cột Load
    if df['Load'].dtype == object:
        df['Load'] = df['Load'].str.replace('%', '').astype(float)

    current_load_percent = lf * 100
    

    results = {}

    for p in pollutants:
        # Lấy tên cột chuẩn trong CSV
        target_col = p

        if target_col not in df.columns:
            print(f"Warning: Pollutant '{p}' (mapped to '{target_col}') not found in {man_type} table. Returning 1.0")
            results[p] = 1.0
            continue

        # 4. Tìm giá trị LAF tại mức tải gần nhất (Nearest Interpolation)
        # Vì bảng MAN rất chi tiết (từng 1%), việc lấy nearest là đủ chính xác
        idx = (df['Load'] - current_load_percent).abs().idxmin()
        val = df.loc[idx, target_col]
        results[p] = float(val)

    return results

# --- TEST CASES ---
print("Test 1 - Low Load (5%) for NOx and HC on C3:")
print(compute_laf_man(['NOx', 'HC'], lf=0.05, man_type='C3'))   
print("\nTest 2 - High Load (80%) for CO on SV:")
print(compute_laf_man(['CO'], lf=0.80, man_type='SV'))


Test 1 - Low Load (5%) for NOx and HC on C3:
{'NOx': 1.72, 'HC': 2.23}

Test 2 - High Load (80%) for CO on SV:
{'CO': 0.6}


In [8]:
def compute_real_ef_man(pollutants, lf, year=2010, rpm=100, man_type='C3'):
    """
    Tính toán EF thực tế cho động cơ MAN.
    Công thức: EF = EF_base * LAF
    (Động cơ MAN thường tích hợp các điều chỉnh khác vào thẳng bảng LAF)
    """
    base_efs = compute_ef_base(pollutants, engine='main', year=year, rpm=rpm)
    
    lafs = compute_laf_man(pollutants, lf, man_type=man_type)
    
    final_ef = {}
    
    
    for p in pollutants:
        b = base_efs.get(p, 0.0)
        l = lafs.get(p, 1.0)
        
        real = b * l
        final_ef[p] = real
        
    print( final_ef)
        
    return final_ef


# --- TEST CASES ---

# Test 1: Động cơ MAN C3, Tải 75%, chạy năm 2010 (Tier 2)
print("TEST 1: MAN C3, Load 75%, Tier 2")
compute_real_ef_man(['NOx', 'CO2'], lf=0.75, year=2010, rpm=100, man_type='C3')

# Test 2: Động cơ MAN SV (Slide Valve), Tải thấp 25%, Tier 2
print("\nTEST 2: MAN SV, Load 25%, Tier 2")
compute_real_ef_man(['NOx', 'HC'], lf=0.25, year=2010, rpm=100, man_type='SV')

TEST 1: MAN C3, Load 75%, Tier 2
{'NOx': 14.544, 'CO2': 587.07}

TEST 2: MAN SV, Load 25%, Tier 2
{'NOx': 17.28, 'HC': 0.44999999999999996}


{'NOx': 17.28, 'HC': 0.44999999999999996}

In [9]:
import pandas as pd
import numpy as np
import compute
# Số lượng dòng
num_rows = 1000

# 1. TẠO CÁC THÔNG SỐ CƠ BẢN
# Loại tàu (theo danh sách bạn cung cấp)
ship_types = [
    'bulk_carrier', 'container_ship', 'cruise_ship', 
    'general_cargo_ship', 'other_ship', 'ro_ro_ship', 
    'reefer_ship', 'oil_tanker'
]
col_type = np.random.choice(ship_types, num_rows)

# Năm đóng, Phao
col_year = np.random.randint(1990, 2025, num_rows)
col_buoy = np.zeros(num_rows, dtype=int) # Luôn là 0

# 2. TẠO DỮ LIỆU THỜI GIAN (GIỜ)
# Tạo ngẫu nhiên 3 thành phần con

# Tính tổng theo yêu cầu: Giờ trong bến = Tổng 3 giờ kia


# 3. TẠO DỮ LIỆU VẬN TỐC (Hải lý/giờ)
# Trip ~ 10 (+- dao động nhẹ)
v_trip = np.round(np.random.normal(10, 1.0, num_rows), 2)
# Đảm bảo không âm
v_trip = np.maximum(v_trip, 0.1)

# Manuever ~ 5 (+- dao động nhẹ)
v_manuever = np.round(np.random.normal(5, 0.5, num_rows), 2)
v_manuever = np.maximum(v_manuever, 0.1)

# V_max > V_trip (Cộng thêm khoảng 5-10 hải lý)
v_max = np.round(v_trip + np.random.uniform(5, 10, num_rows), 2)

t_neo = np.round(np.random.uniform(10, 24, num_rows), 2)           # Neo: 1-24h
t_hanh_trinh = np.round([compute.compute_A(v, 0, 'trip') for v in v_trip], 2)  # Hành trình: 24-100h
t_dieu_dong = np.round([compute.compute_A(v, 0, 'maneuver') for v in v_manuever], 2)      # Điều động: 1-5h

t_trong_cang = t_neo + t_hanh_trinh + t_dieu_dong

# 4. TẠO DỮ LIỆU ĐỘNG CƠ & CÔNG SUẤT
# Công suất (kW) - Random tùy ý trong khoảng hợp lý
p_main = np.round(np.random.uniform(5000, 50000, num_rows), 1)
p_aux = np.round(p_main * 0.2, 1) # Máy phụ khoảng 20% máy chính

# Thuộc tính động cơ
col_is_man = np.random.choice([True, False], num_rows)
col_valve_type = np.random.choice(['C3', 'SV'], num_rows)
col_rpm = np.round(np.random.uniform(50, 1999, num_rows), 1) # Nhỏ hơn 2000

# 5. TỔNG HỢP DATAFRAME
df = pd.DataFrame({
    'type': col_type,              # Loại tàu
    'year': col_year,              # Năm đóng
    'buoy': col_buoy,              # Phao (=0)
    
    # Nhóm Thời gian
    'time_port': t_trong_cang,     # Tổng 3 cái kia
    'time_anchor': t_neo,
    'time_trip': t_hanh_trinh,
    'time_maneuver': t_dieu_dong,
    
    # Nhóm Vận tốc
    'v_trip': v_trip,              # ~ 10
    'v_maneuver': v_manuever,      # ~ 5
    'v_max': v_max,                # > v_trip
    
    # Nhóm Công suất
    'P_main': p_main,
    'P_aux': p_aux,
    
    # Thông số kỹ thuật
    'is_man': col_is_man,
    'valve_type': col_valve_type,
    'rpm': col_rpm
})

# Lưu file
filename = 'data/Input_Ship_Data_Final.csv'
df.to_csv(filename, index=False)

print(f"Đã tạo file '{filename}' thành công.")
print("5 dòng đầu tiên:")
print(df.head())

Đã tạo file 'data/Input_Ship_Data_Final.csv' thành công.
5 dòng đầu tiên:
           type  year  buoy  time_port  time_anchor  time_trip  time_maneuver  \
0    oil_tanker  1992     0      19.04        12.87       0.23           5.94   
1   cruise_ship  2007     0      23.73        18.50       0.21           5.02   
2  bulk_carrier  2007     0      29.66        23.12       0.21           6.33   
3    other_ship  2000     0      28.68        21.28       0.24           7.16   
4    other_ship  2005     0      21.60        15.33       0.19           6.08   

   v_trip  v_maneuver  v_max   P_main   P_aux  is_man valve_type     rpm  
0   10.16        5.10  19.93  27346.9  5469.4    True         SV  1758.4  
1   11.19        6.03  18.87   8359.5  1671.9   False         C3   810.9  
2   11.32        4.78  17.50  24410.1  4882.0   False         C3   729.3  
3    9.95        4.23  16.80  34350.6  6870.1    True         C3  1934.0  
4   12.38        4.98  18.08  44630.2  8926.0   False         C3

In [ ]:
import pandas as pd
import numpy as np
import compute  # Giả định bạn đã lưu các hàm compute vào file compute.py

# ==========================================
# PHẦN 1: TẠO DỮ LIỆU ĐẦU VÀO (1000 DÒNG)
# ==========================================
print("Đang tạo dữ liệu đầu vào...")
num_rows = 1000

# 1. Các thông số cơ bản
ship_types = [
    'bulk_carrier', 'container_ship', 'cruise_ship', 
    'general_cargo_ship', 'other_ship', 'ro_ro_ship', 
    'reefer_ship', 'oil_tanker'
]
col_type = np.random.choice(ship_types, num_rows)
col_year = np.random.randint(1990, 2025, num_rows)
col_buoy = np.zeros(num_rows, dtype=int)

# 2. Vận tốc (Hải lý/giờ)
v_trip = np.round(np.maximum(np.random.normal(10, 1.0, num_rows), 0.1), 2)
v_maneuver = np.round(np.maximum(np.random.normal(5, 0.5, num_rows), 0.1), 2)
v_max = np.round(v_trip + np.random.uniform(5, 10, num_rows), 2)

# 3. Thời gian (Giờ) - Sử dụng List Comprehension để tránh lỗi TypeError
t_neo = np.round(np.random.uniform(10, 24, num_rows), 2)

# Gọi hàm compute_A cho từng phần tử trong mảng v_trip/v_maneuver
t_hanh_trinh = np.array([compute.compute_A(v, 0, 'trip') for v in v_trip])
t_hanh_trinh = np.round(t_hanh_trinh, 2)

t_dieu_dong = np.array([compute.compute_A(v, 0, 'maneuver') for v in v_maneuver])
t_dieu_dong = np.round(t_dieu_dong, 2)

t_trong_cang = t_neo + t_hanh_trinh + t_dieu_dong

# 4. Công suất (kW) & Động cơ
p_main = np.round(np.random.uniform(5000, 50000, num_rows), 1)
p_aux = np.round(p_main * 0.2, 1)

col_is_man = np.random.choice([True, False], num_rows)
col_valve_type = np.random.choice(['C3', 'SV'], num_rows)
col_rpm = np.round(np.random.uniform(50, 1999, num_rows), 1)

# Tạo DataFrame ban đầu
df = pd.DataFrame({
    'type': col_type,
    'year': col_year,
    'buoy': col_buoy,
    'time_port': t_trong_cang,
    'time_anchor': t_neo,
    'time_trip': t_hanh_trinh,
    'time_maneuver': t_dieu_dong,
    'v_trip': v_trip,
    'v_maneuver': v_maneuver,
    'v_max': v_max,
    'P_main': p_main,
    'P_aux': p_aux,
    'is_man': col_is_man,
    'valve_type': col_valve_type,
    'rpm': col_rpm
})

# ==========================================
# PHẦN 2: TÍNH TOÁN PHÁT THẢI (5 CỘT E)
# ==========================================
print("Đang tính toán phát thải (E1 -> E5)...")

# Danh sách 5 chất ô nhiễm cần tính
pollutants_list = ['PM10', 'PM2.5', 'NOx', 'SOx', 'CO', 'HC', 'CO2', 'N2O', 'CH4']

def calculate_total_emission_per_row(row):
    """
    Hàm tính tổng E = E1 + E2 + E3 + E4 + E5 cho từng dòng
    Dựa trên công thức trong ảnh.
    """
    # Lấy dữ liệu từ dòng hiện tại
    v_t = row['v_trip']
    v_m = row['v_maneuver']
    v_max = row['v_max']
    p_main = row['P_main']
    p_aux = row['P_aux']
    t_trip = row['time_trip']   # A hành trình
    t_man = row['time_maneuver'] # A điều động
    t_anchor = row['time_anchor'] # A neo đậu
    
    # Các tham số kỹ thuật
    is_man = row['is_man']
    ship_type = row['type']
    year = row['year']
    rpm = row['rpm']
    valve = row['valve_type']

    # --- BƯỚC 1: TÍNH LOAD FACTOR (LF) ---
    # LF Máy chính
    lf_main_trip = compute.compute_lf(v_t, v_max, engine='main', type=ship_type, status='trip')
    lf_main_man = compute.compute_lf(v_m, v_max, engine='main', type=ship_type, status='maneuver')
    
    # LF Máy phụ (Đọc từ CSV hoặc default)
    lf_aux_trip = compute.compute_lf(v_t, v_max, engine='auxiliary', type=ship_type, status='trip')
    lf_aux_man = compute.compute_lf(v_m, v_max, engine='auxiliary', type=ship_type, status='maneuver')
    
    # Lưu ý: Công thức trong ảnh E5 ghi "LF điều động" cho phần Neo đậu.
    # Tuy nhiên, để chính xác logic vật lý, ta thường dùng LF Mooring cho Neo đậu.
    # Ở đây tôi dùng LF mooring (neo đậu) cho chuẩn xác, nếu muốn giống hệt chữ trong ảnh thì đổi thành status='maneuver'.
    lf_aux_anchor = compute.compute_lf(0, v_max, engine='auxiliary', type=ship_type, status='maneuver')

    # --- BƯỚC 2: TÍNH EMISSION FACTOR (EF) ---
    # Cần tính EF riêng cho từng trạng thái vì EF phụ thuộc vào LF
    
    # Hàm chọn loại động cơ (MAN hoặc Non-MAN)
    calc_ef_func = compute.compute_real_ef_man if is_man else compute.compute_real_ef_non_man
    
    # EF Máy chính
    ef_main_trip_dict = calc_ef_func(pollutants_list, lf=lf_main_trip, engine='main', year=year, rpm=rpm, valve_type=valve)
    ef_main_man_dict = calc_ef_func(pollutants_list, lf=lf_main_man, engine='main', year=year, rpm=rpm, valve_type=valve)
    
    # EF Máy phụ (Luôn là non-man hoặc tùy logic, ở đây giả sử tuân theo is_man của tàu hoặc mặc định non-man)
    # Thường máy phụ là Non-MAN 4 kỳ, ta dùng compute_real_ef_non_man cho máy phụ
    ef_aux_trip_dict = compute.compute_real_ef_non_man(pollutants_list, lf=lf_aux_trip, engine='auxiliary', year=year, rpm=rpm)
    ef_aux_man_dict = compute.compute_real_ef_non_man(pollutants_list, lf=lf_aux_man, engine='auxiliary', year=year, rpm=rpm)
    ef_aux_anchor_dict = compute.compute_real_ef_non_man(pollutants_list, lf=lf_aux_anchor, engine='auxiliary', year=year, rpm=rpm)

    # --- BƯỚC 3: TÍNH E1 -> E5 VÀ TỔNG HỢP ---
    results = {}
    
    for p in pollutants_list:
        ef_m_t = ef_main_trip_dict.get(p, 0)
        ef_m_m = ef_main_man_dict.get(p, 0)
        ef_a_t = ef_aux_trip_dict.get(p, 0)
        ef_a_m = ef_aux_man_dict.get(p, 0)
        ef_a_a = ef_aux_anchor_dict.get(p, 0)
        
        # Tính toán từng thành phần E
        E1 = p_main * t_trip * lf_main_trip * ef_m_t    # Main - Trip
        E2 = p_main * t_man * lf_main_man * ef_m_m      # Main - Maneuver
        E3 = p_aux * t_trip * lf_aux_trip * ef_a_t      # Aux - Trip
        E4 = p_aux * t_man * lf_aux_man * ef_a_m        # Aux - Maneuver
        E5 = p_aux * t_anchor * lf_aux_anchor * ef_a_a  # Aux - Anchor
        
        # --- QUAN TRỌNG: LƯU TẤT CẢ VÀO DICTIONARY ---
        results[f'E1_{p}'] = E1
        results[f'E2_{p}'] = E2
        results[f'E3_{p}'] = E3
        results[f'E4_{p}'] = E4
        results[f'E5_{p}'] = E5
        results[f'Total_{p}'] = E1 + E2 + E3 + E4 + E5

    return pd.Series(results)

# Áp dụng
emission_cols = df.apply(calculate_total_emission_per_row, axis=1)
df_final = pd.concat([df, emission_cols], axis=1)

# ==========================================
# PHẦN 3: XUẤT FILE
# ==========================================
filename = 'data/Ship_Data_Full_Details.csv'
df_final.to_csv(filename, index=False)

print(f"Xong! File '{filename}' đã được lưu.")
print("Các cột mới bao gồm (ví dụ cho NOx): E1_NOx, E2_NOx, ..., Total_NOx")
print(df_final[['type', 'E1_NOx', 'E2_NOx', 'Total_NOx']].head())

Đang tạo dữ liệu đầu vào...
Đang tính toán phát thải (E1 -> E5)...
2
{'PM10': 0.19448000000000001, 'PM2.5': 0.17887999999999998, 'NOx': 2.652, 'SOx': 0.41303, 'CO': 0.53724, 'HC': 0.555, 'CO2': 676.71, 'N2O': 0.029580000000000002, 'CH4': 0.0111}
2
{'PM10': 0.80971, 'PM2.5': 0.74476, 'NOx': 7.592, 'SOx': 0.9824500000000002, 'CO': 3.12664, 'HC': 5.84, 'CO2': 1603.08, 'N2O': 0.08468, 'CH4': 0.1168}
1
{'PM10': 0.19, 'PM2.5': 0.17, 'NOx': 2.0, 'SOx': 0.42, 'CO': 1.1, 'HC': 0.4, 'CO2': 696.0, 'N2O': 0.029, 'CH4': 0.008}
1
{'PM10': 0.19, 'PM2.5': 0.17, 'NOx': 2.0, 'SOx': 0.42, 'CO': 1.1, 'HC': 0.4, 'CO2': 696.0, 'N2O': 0.029, 'CH4': 0.008}
1
{'PM10': 0.19, 'PM2.5': 0.17, 'NOx': 2.0, 'SOx': 0.42, 'CO': 1.1, 'HC': 0.4, 'CO2': 696.0, 'N2O': 0.029, 'CH4': 0.008}
1
{'PM10': 0.187, 'PM2.5': 0.172, 'NOx': 12.2, 'SOx': 0.401, 'CO': 1.1, 'HC': 0.5, 'CO2': 657.0, 'N2O': 0.029, 'CH4': 0.01}
2
{'PM10': 0.80971, 'PM2.5': 0.74476, 'NOx': 35.623999999999995, 'SOx': 0.9824500000000002, 'CO': 4.19254, 'HC': 5